# PETs/TETs – Hyperledger Aries / PySyft – Manufacturer 3 (Holder) 💼

In [ ]:
%%javascript
document.title = '💼 Manufacturer 3 Agent'

## PART 3: Connect with City to Analyze Data

**What:** -

**Why:** -

**How:** <br>


**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_manufacturers.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [ ]:
import libs.helpers as helpers
import os
import syft as sy
import time
import torch
import sympc

from aries_cloudcontroller import AriesAgentController
from libs.agent_connection_manager import CredentialHolder
from pprintpp import pprint
from termcolor import colored
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

#### 0.2 – Variables

In [ ]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer3 Agent
#### 1.1 – Init ACA-PY agent controller

In [ ]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [ ]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

#### 1.3 – Init ACM issuing authority

In [ ]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer3_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer3_agent.get_credentials()

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the Manufacturer3 requests a connection with the Authority to be certified as an official city agency. Thus, the Manufacturer3 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [ ]:
# Variables
alias = "undisclosedM3"
auto_accept = True
label = None

# Receive connection invitation
connection_id = manufacturer3_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

<div style="font-size: 25px"><center><b>Break Point 2/3/4</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.3</b></center>

---

### 3 – Create Presentation to Send Proof Presentation

#### 3.1 – Create presentation that satisfies requirements of proof request
Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. To do so, the `prepare_presentation()` function runs through the following steps: 
1. Get all proof requests that were sent through `connection_id`
2. Get the most recent `presentation_exchange_id` and the corresponding `proof_request` from (1)
3. Get the restrictions the City agent defined in `proof_request` from (2)
4. Compare all VCs the Manufacturer3 agent has stored, and find (if available) a VC that satisfies the restrictions from (3)
5. Return a presentation dictionary from a VC from (4) that satisfies all requirements. Generally, a presentation consists of three classes of attributes: <br>
a. `requested_attributes`: Attributes that were signed by an issuer and have been revealed in the presentation process <br>
b. `self_attested_attributes`: Attributes that the prover has self attested to in the presentation object. <br>
c. `requested_predicates` (predicate proofs): Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

In [ ]:
presentation, presentation_exchange_id = manufacturer3_agent.prepare_presentation(connection_id)
presentation

#### 3.2 – Send Presentation

Send the presentation to the recipient of `presentation_exchange_id`

In [ ]:
manufacturer3_agent.send_proof_presentation(presentation_exchange_id, presentation)

<div style="font-size: 25px"><center><b>Break Point 6/7/8</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 3 and 4.1 <br> If you have sent proof presentations from all manufacturers, proceed to the City Notebook's Step 3.3 </b></center>


---

### 4 – Do Data Science

#### 4.1 – Establish a Duet Connection: Send Duet invitation
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [ ]:
# Set up connection_id to use for duet connection
manufacturer3_agent._update_connection(connection_id=connection_id, is_duet_connection=True, reset_duet=True)

# Create duet invitation for city agent
duet = sy.launch_duet(credential_exchanger=manufacturer3_agent)

#### 4.2 - Load data to duet store

In [ ]:
# Step 8: test there is no data in the server atm
duet.store.pandas

In [ ]:
age_data = torch.tensor([50,23,72,83])
age_data = age_data.tag("test_data_age")
age_data = age_data.describe("description for age_data")
age_data_pointer = age_data.send(duet, pointable=True)

In [ ]:
duet.store.pandas

In [ ]:
# Step 11: Authorize the requests ds can make
duet.requests.add_handler(name="reconstruct", action="accept")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()